# Automation to create several Reports in a loop !

In [2]:
import pickle
from myLibrary import Experiment
from myLibrary import DataProcessor as DP
import os

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import matplotlib.backends.backend_pdf as pdf_backend
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
def create_charts_to_pdf(df, title, metadata, filename):
    base_features = set()

    for column in df.columns:
        base_feature = column.replace('_pred', '')
        base_features.add(base_feature)

    with pdf_backend.PdfPages(filename) as pdf:
        # Add metadata to the beginning of the PDF
        fig = plt.figure(figsize=(6, 10))
        plt.title('Test Metadata')
        plt.figtext(0.5, 0.05, metadata, ha='center', fontsize=8)
        #plt.figtext(0.1, 0.9, metadata)  # Adjust the coordinates as needed
        plt.axis('off')
        pdf.savefig(fig)
        plt.close(fig)

        MAE = {}
        MSE = {}
        for base_feature in base_features:
            pred_feature = f'{base_feature}_pred'

            fig = plt.figure(figsize=(10, 6))
            plt.plot(df.index, df[base_feature], label=base_feature, color='#1f77b4')
            plt.plot(df.index, df[pred_feature], label=pred_feature, color='#2ca02c')

            plt.xlabel('Time')
            plt.ylabel(base_feature)
            plt.title(f'{title}')
            plt.legend()

            # Calculate metrics
            if base_feature.startswith("WDIR"):
                mae = DP.mean_absolute_error_WDIR(df[base_feature], df[pred_feature])
                mse = DP.mean_square_error_WDIR(df[base_feature], df[pred_feature])
            else:
                mae = mean_absolute_error(df[base_feature], df[pred_feature])
                mse = mean_squared_error(df[base_feature], df[pred_feature])
            MAE[base_feature] = mae.round(3)
            MSE[base_feature] = mse.round(3)

            # Add metrics to the current page of the PDF
            fig.text(0.01, 0.02, f"MAE: {mae.round(3)}\nMSE: {mse.round(3)}")  # Adjust the coordinates as needed
            pdf.savefig(fig)
            plt.close(fig)

        return MAE, MSE

In [4]:
def create_error_table(dictionary):
    # Extract unique measurements and station IDs
    measurements = set()
    station_ids = set()
    for key in dictionary.keys():
        measurement, station_id, *_ = key.split('_')
        measurements.add(measurement)
        station_ids.add(station_id)

    # Create empty table
    table = pd.DataFrame(index=sorted(station_ids), columns=sorted(measurements))

    # Fill table with dictionary values
    for key, value in dictionary.items():
        measurement, station_id, *_ = key.split('_')
        if key.endswith('ERA5'):
            measurement += '_ERA5'
        table.loc[station_id, measurement] = value

    # Replace missing values with NaN
    table = table.fillna(np.nan)

    return table

In [5]:
def evaluation(filename):

    # open the file for reading in binary mode
    report: Experiment
    with open(f'data/reports/{filename}', 'rb') as f:
        # load the object from the file using pickle.load()
        report = pickle.load(f)

    #Create directory if not exists
    filename = f"{filename.replace('.pickle','')}"
    directory = f"data/evaluation/{filename}"
    if not os.path.exists(directory):
        os.makedirs(directory)  # Create the directory

   #Create PDF report
    title =  f"One-Shot-Forecast of: {filename.replace('.pickle','')}"
    MAE_dict, MSE_dict = create_charts_to_pdf(report.one_shot_forecast, title, report.get_settings_string(), f"{directory}/{filename}_charts.pdf")

    #Create MAE CSV File
    MAE_df = create_error_table(MAE_dict)
    MAE_df.to_csv(f"{directory}/{filename}_MAE.csv")

    #Create MSE CSV File
    MSE_df = create_error_table(MSE_dict)
    MSE_df.to_csv(f"{directory}/{filename}_MSE.csv")

    print(f"Saved: {directory}/{filename}")

In [7]:
approach = "MLM"   # "MLM" ... Multi Location Modelling, "SSUM" ... Station Specific Unified Model
model_names = ["LSTM", "GRU", "CNN", "TCN"]
alpha_values = np.arange(0, 1.1, 0.1)

if approach == "MLM":
    for model in model_names:
        for alpha in alpha_values:
            filename = f'report_A_{model}_{format(alpha, ".1f")}.pickle'
            evaluation(filename)

elif approach == "SSUM":
    for model in model_names:
        for alpha in alpha_values:
            filename = f'report_B_{model}_{format(alpha, ".1f")}'
            evaluation(filename)

Saved: data/evaluation/report_A_LSTM_0.0/report_A_LSTM_0.0
Saved: data/evaluation/report_A_LSTM_0.1/report_A_LSTM_0.1
Saved: data/evaluation/report_A_LSTM_0.2/report_A_LSTM_0.2
Saved: data/evaluation/report_A_LSTM_0.3/report_A_LSTM_0.3
Saved: data/evaluation/report_A_LSTM_0.4/report_A_LSTM_0.4
Saved: data/evaluation/report_A_LSTM_0.5/report_A_LSTM_0.5
Saved: data/evaluation/report_A_LSTM_0.6/report_A_LSTM_0.6
Saved: data/evaluation/report_A_LSTM_0.7/report_A_LSTM_0.7
Saved: data/evaluation/report_A_LSTM_0.8/report_A_LSTM_0.8
Saved: data/evaluation/report_A_LSTM_0.9/report_A_LSTM_0.9
Saved: data/evaluation/report_A_LSTM_1.0/report_A_LSTM_1.0
Saved: data/evaluation/report_A_GRU_0.0/report_A_GRU_0.0
Saved: data/evaluation/report_A_GRU_0.1/report_A_GRU_0.1
Saved: data/evaluation/report_A_GRU_0.2/report_A_GRU_0.2
Saved: data/evaluation/report_A_GRU_0.3/report_A_GRU_0.3
Saved: data/evaluation/report_A_GRU_0.4/report_A_GRU_0.4
Saved: data/evaluation/report_A_GRU_0.5/report_A_GRU_0.5
Saved: da